# Get historical data throgh API 
# Make Api call for crypto and stock(S&P500) from Allpaca
# Make API call for comodities from barchart


In [2]:
# Limit the number of securities for each alloocation( depends on the data fed)

# Get combination of alocated assets and weighting 
# Use KMeans algorithm for clustering 
from sklearn.cluster import KMeans
import numpy as np
import requests
import json

# Data collection and preparation for Stocks, Crypto, and Commodities

In [3]:
# Using NASDAQ Data Link API to collect commodities data from the OPEC Crude Oil Basket.
# Collecting 5-year crude oil market data from OPEC.
OPEC_url = "https://data.nasdaq.com/api/v3/datasets/OPEC/ORB.json?api_key=9MXjsAzYyYicz7jtVQX5&start_date=2018-02-06"
OPEC_response = requests.get(OPEC_url)
OPEC_data = OPEC_response.json()

print(json.dumps(OPEC_data, indent=4, sort_keys=True))

{
    "dataset": {
        "collapse": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Value"
        ],
        "data": [
            [
                "2023-02-07",
                80.5
            ],
            [
                "2023-02-06",
                78.2
            ],
            [
                "2023-02-03",
                79.36
            ],
            [
                "2023-02-02",
                80.43
            ],
            [
                "2023-02-01",
                82.82
            ],
            [
                "2023-01-31",
                81.95
            ],
            [
                "2023-01-30",
                83.45
            ],
            [
                "2023-01-27",
                85.31
            ],
            [
                "2023-01-26",
                84.47
            ],
            [
                "2023-01-25",
                84.59
            ],
            [
        

In [4]:
# Using NASDAQ Data Link API to collect commodities data from the LBMA London Bullion Market Association.
# Collecting 5-year gold market data from LBMA.
LBMA_gold_url = "https://data.nasdaq.com/api/v3/datasets/LBMA/GOLD.json?api_key=9MXjsAzYyYicz7jtVQX5&start_date=2013-02-07"
LBMA_gold_response = requests.get(LBMA_gold_url)
LBMA_gold_data = LBMA_gold_response.json()

print(json.dumps(LBMA_gold_data, indent=4, sort_keys=True))

{
    "dataset": {
        "collapse": null,
        "column_index": null,
        "column_names": [
            "Date",
            "USD (AM)",
            "USD (PM)",
            "GBP (AM)",
            "GBP (PM)",
            "EURO (AM)",
            "EURO (PM)"
        ],
        "data": [
            [
                "2023-02-08",
                1880.75,
                1872.65,
                1555.76,
                1552.61,
                1750.04,
                1747.0
            ],
            [
                "2023-02-07",
                1873.8,
                1870.7,
                1561.81,
                1561.66,
                1748.26,
                1751.16
            ],
            [
                "2023-02-06",
                1873.5,
                1873.25,
                1556.69,
                1554.61,
                1739.54,
                1743.43
            ],
            [
                "2023-02-03",
                1910.0,
                1

In [11]:
# Calculate the returns for each asset
# crypto_returns.pct_change()
# stocks_returns.pct_change()
commodity.pct_change()

AttributeError: 'dict' object has no attribute 'pct_change'

In [ ]:
#Run montecarlo model

In [10]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_alloc = int(input('max equity'))
        break
    except:
        print('thats not an integer')


max equity 30


In [11]:
#Ask user how much is the allocation for Commodity Allocation
# Check the input 
while True:
    try:
        commodity_alloc = int(input('max commodity'))
        break
    except:
        print('thats not an integer')


max commodity 20


In [13]:
#Ask user how much is the allocation for Crypto Allocation
# Check the input 
while True:
    try:
        crypto_alloc = int(input('max crypto'))
        break
    except:
        print('thats not an integer')


max crypto 50


In [16]:
#Data from 3 assets with their weight amounts
#Limit each equity t0 allocate 10 items 
assets = {'equity':range(0,equity_alloc,10),'commodity':range(0,commodity_alloc,10),'crypto':range(0,crypto_alloc,10)}




NameError: name 'crypto_alloc' is not defined

In [6]:
#Allowing only 3 cluster of assets 
kmeans = KMeans(n_clusters=3)



In [7]:
#Fit the model to the asset data 
kmeans.fit(assets)



TypeError: float() argument must be a string or a number, not 'dict'

In [ ]:
# Plot vizualization fort the clusters 

In [ ]:
# Ru